In [2]:
#Necessary Imports
# !pip install ta
import numpy as np
import pandas as pd
import os
import pickle
import torch
# import ta
from google.colab import drive
drive.mount('/content/drive')

#Data Source Location
os.chdir("/content/drive/My Drive/mining-on-stock")

Mounted at /content/drive


In [ ]:
#Load Feature Engineered Stock dataset: 100 Features including labels
with open('data_with_features_clean_train.pkl', 'rb') as train_file: #To do: Change file name.
  train_data = pickle.load(train_file)

In [ ]:
len(train_data.columns)

101

In [ ]:
#Last 5 data points of the first stock
print(f'Last 5 datapoints of the first stock among 2000 stocks')
stock_len = 2201
train_data.iloc[stock_len-5: stock_len] #To do: Once the concatenated file is ready.

Last 5 datapoints of the first stock among 2000 stocks


,Open,High,Low,Close,Volume,awesome_oscillator (AO),bollinger_percent_b (percent_b),cfo (cfo),cmo (cmo),detrended_price_oscillator (dpo),...,ichimoku_clouds (senkou_span_a),ichimoku_clouds (chikou_span),keltner_channel (Keltner Center),keltner_channel (Keltner Upper),keltner_channel (Keltner Lower),linear_reg_forecast (LRF),mass_index (Mass_Index),median_price (Median Price),money_flow_index (Money Flow Index),Label
2197,0.680849,0.683633,0.672997,0.674055,0.351048,0.010157,0.744441,0.068595,21.168686,0.004813,...,0.683299,0.689504,0.677062,0.692473,0.661650,0.689504,1.080827,0.678315,7.843752,decrease
2198,0.674389,0.674890,0.671326,0.674055,0.143069,0.010020,0.723160,0.068595,19.300231,0.005677,...,0.682909,0.689504,0.677206,0.692105,0.662307,0.689504,0.966600,0.673108,7.179743,no big change
2199,0.674110,0.676227,0.672440,0.676004,0.312048,0.010509,0.755154,0.068595,21.857938,0.008792,...,0.682505,0.689504,0.677414,0.691979,0.662849,0.689504,0.896525,0.674333,7.179743,increase
2200,0.675725,0.675837,0.668820,0.669321,0.279230,0.008306,0.536443,0.068595,18.491048,0.003247,...,0.682505,0.689504,0.677475,0.692413,0.662537,0.689504,0.925453,0.672328,7.535366,decrease
2201,0.669293,0.671326,0.662973,0.664198,0.552933,0.005024,0.362143,0.068595,15.904361,-0.000906,...,0.679735,0.689504,0.677201,0.691448,0.662953,0.689504,0.977753,0.667149,7.707911,decrease


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import multiprocessing

class MyDataset(Dataset):
    def __init__(self, df, seq_length):
        self.X = []
        self.y = []
        
        labels = df.iloc[:, -1].values # Extract labels from the last column
        cnt = len(labels)
        
        df = df.iloc[:, :-1] # Remove labels column from the DataFrame        
        values = torch.tensor(df.values) # Convert DataFrame to tensor
        stock_len = 2201
        # Create input/output sequences
        while cnt != 0:
          for i in range(seq_length, stock_len):
            self.X.append(values[i-seq_length:i, :])
            self.y.append(labels[i])
          cnt -= 2201
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

def data_generator(train_data, seq_length, batch_size, num_workers):
    train_dataset = MyDataset(train_data, seq_length)    
    
    # Create DataLoader
    # Optimzation 1: Dataset and Dataloaders for batch processing
    # Optimization 2: (num_of_workers, pin_memory(GPU specific))    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    for X_batch, y_batch in train_loader:
        # Convert labels to tensor
        y_train = np.array(y_batch)
        labels_encoded = torch.zeros((len(y_train), 3), dtype=torch.float32)
        labels_encoded[y_train == 'increase', 2] = 1
        labels_encoded[y_train == 'no big change', 1] = 1
        labels_encoded[y_train == 'decrease', 0] = 1
        y_train_tensor = torch.tensor(labels_encoded).float().clone().detach()

        yield X_batch, y_train_tensor

    # Free up memory
    del train_dataset, train_loader
    del labels_encoded
    del y_train, X_batch, y_batch, y_train_tensor

def prepare_train_data(train_data = None, seq_length = 100, batch_size = 32):
  num_workers=multiprocessing.cpu_count() #parallel computing

  # Optimization 4 (Using data generator)
  generator = data_generator(train_data, seq_length, batch_size, num_workers) 

  return generator

In [ ]:
train_data[:2201]

,Open,High,Low,Close,Volume,awesome_oscillator (AO),bollinger_percent_b (percent_b),cfo (cfo),cmo (cmo),detrended_price_oscillator (dpo),...,ichimoku_clouds (senkou_span_a),ichimoku_clouds (chikou_span),keltner_channel (Keltner Center),keltner_channel (Keltner Upper),keltner_channel (Keltner Lower),linear_reg_forecast (LRF),mass_index (Mass_Index),median_price (Median Price),money_flow_index (Money Flow Index),Label
1,0.672106,0.674597,0.670546,0.672106,0.208023,0.005725,0.464494,1.376252,38.508723,0.000000,...,0.684928,0.689504,0.679429,0.695254,0.674792,0.689504,1.004993,0.672572,27.384836,no big change
2,0.672106,0.678621,0.671827,0.676282,0.142405,0.005725,0.464494,0.944498,38.508723,0.000000,...,0.684928,0.689504,0.679482,0.695254,0.674792,0.689504,1.004993,0.675224,27.384836,increase
3,0.676338,0.677340,0.672384,0.673498,0.146400,0.005725,0.464494,1.546220,38.508723,0.000000,...,0.684928,0.689504,0.679507,0.695254,0.674792,0.689504,1.004993,0.674862,27.384836,decrease
4,0.673442,0.676004,0.672997,0.675892,0.116960,0.005725,0.464494,1.346294,38.508723,0.000000,...,0.684928,0.689504,0.679506,0.695254,0.674792,0.689504,1.004993,0.674500,27.384836,increase
5,0.675892,0.676115,0.673609,0.674890,0.205266,0.005725,0.464494,1.667214,38.508723,0.000000,...,0.684928,0.689504,0.679529,0.695254,0.674792,0.689504,1.004993,0.674862,27.384836,no big change
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2197,0.680849,0.683633,0.672997,0.674055,0.351048,0.010157,0.744441,0.068595,21.168686,0.004813,...,0.683299,0.689504,0.677062,0.692473,0.661650,0.689504,1.080827,0.678315,7.843752,decrease
2198,0.674389,0.674890,0.671326,0.674055,0.143069,0.010020,0.723160,0.068595,19.300231,0.005677,...,0.682909,0.689504,0.677206,0.692105,0.662307,0.689504,0.966600,0.673108,7.179743,no big change
2199,0.674110,0.676227,0.672440,0.676004,0.312048,0.010509,0.755154,0.068595,21.857938,0.008792,...,0.682505,0.689504,0.677414,0.691979,0.662849,0.689504,0.896525,0.674333,7.179743,increase
2200,0.675725,0.675837,0.668820,0.669321,0.279230,0.008306,0.536443,0.068595,18.491048,0.003247,...,0.682505,0.689504,0.677475,0.692413,0.662537,0.689504,0.925453,0.672328,7.535366,decrease


In [ ]:
gen = MyDataset(train_data[0:2201],100)
gen = None
for i, j in data_generator(train_data[:2201], 100, 16, multiprocessing.cpu_count()):
  print(i.shape, j.shape)


<ipython-input-5-d26e89e12f47>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(labels_encoded).float().clone().detach()


torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([16, 100, 100]) torch.Size([16, 3])
torch.Size([1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score
from tqdm import tqdm

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True) #num_layers = 2
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)
        
        # Use Xavier initialization for weights
        init.xavier_uniform_(self.lstm.weight_ih_l0)
        init.orthogonal_(self.lstm.weight_hh_l0)
        init.constant_(self.lstm.bias_ih_l0, 0.0)
        init.constant_(self.lstm.bias_hh_l0, 0.0)
        init.xavier_uniform_(self.fc.weight)
        init.constant_(self.fc.bias, 0.0)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).requires_grad_().to(device)
        c0 = torch.zeros(1, x.size(0), self.hidden_dim).requires_grad_().to(device)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, 3)
        out = self.softmax(out)
        return out

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def train_model(batch_generator):
  # Evaluate each model for 10 epochs
  # Initialize the model
  learning_rate = 0.001
  num_epochs = 10
  input_dim = 100
  hidden_dim =  64 
  # output_dim = y_train_tensor.shape[1]
  output_dim = 3
  num_models = 1
  # Define the loss function and optimizer
  criterion = nn.CrossEntropyLoss()
  for i in range(num_models):
    # print('Model no', i+1)

    # Initialize the model
    model = LSTM(input_dim, hidden_dim, output_dim).to(device)

    # Define the optimizer and scheduler for the current model
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Checkpointing parameters for the current model
    checkpoint_interval = 1
    checkpoint_file = f'lstm_checkpoint_{i}.pth'

    # Define early stopping parameters
    best_loss = float('inf')
    early_stop_counter = 0
    early_stop_patience = 3

    # Train the current model for 10 epochs
    for epoch in range(num_epochs):
      # Set model to train mode
      model.train()
      # model.double()

      # Iterate over batches
      for x_batch, y_batch in tqdm(generator):
          # Move data to device
          x_batch, y_batch = x_batch.to(device), y_batch.to(device)
          # Cast x_batch to float32
          x_batch = x_batch.float()
          print(x_batch.shape)

          # Forward pass
          outputs = model(x_batch)
          loss = criterion(outputs, y_batch)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      # Print the loss for every 10 epochs
      if (epoch + 1) % 10 == 0:
          print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))

      # Check if current model has reached early stopping criteria
      print(loss.item())
      if loss.item() < best_loss:
          best_loss = loss.item()
          early_stop_counter = 0
      else:
          early_stop_counter += 1

      # if early_stop_counter >= early_stop_patience:
      #     print(f"Training for model {i} stopped early at epoch {epoch+1} due to early stopping")
      #     break

      # Save checkpoint at regular intervals
      if (epoch + 1) % checkpoint_interval == 0:
          torch.save({
              'epoch': epoch + 1,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss.item(),
          }, checkpoint_file)

      # Step the scheduler
      scheduler.step()

    # Delete variables to free up memory
    del model, optimizer, scheduler

    # Empty the cache to free up GPU memory
    torch.cuda.empty_cache()


In [ ]:
len(train_data.iloc[:2201*20, :])


44020

In [ ]:
seq_length = 100 # Define sequence length
batch_size = 64 # Create data loader

# Concatenate tensors of empty(Provding early shape can provide performance improvement)
start = 0
# if start == 0:
# X_train_tensor = torch.empty((batch_size*160, seq_length, 100), dtype=torch.float32)
# y_train_tensor = torch.empty((batch_size*160, 3), dtype=torch.float32)
generator = prepare_train_data(train_data.iloc[:2201*200, :], seq_length, batch_size)
# while start < batch_size*160:
# Prepare the data
# start = iter_generator(generator, start)
# print(start)

In [ ]:
train_model(generator)

In [ ]:
y_batch.shape

torch.Size([21010, 3])

In [ ]:
#Creating n subsets of data to train n models on them. This will help to avoid memory issue and all the models will be able to capture different patterns of data.
def create_subset(X, y, num_models):
  X_subsets = torch.chunk(X, num_models, dim=0)
  y_subsets = torch.chunk(y, num_models, dim=0)
  return X_subsets, y_subsets

In [ ]:
type(X_train_tensor), X_train_tensor.shape, type(y_train_tensor), y_train_tensor.shape

(torch.Tensor,
 torch.Size([336160, 100, 100]),
 torch.Tensor,
 torch.Size([336160, 3]))

In [ ]:
print(X_train_split.shape), print(X_val_split.shape), print(y_train_split.shape), print(y_val_split.shape)

torch.Size([868400, 30, 10])
torch.Size([217100, 30, 10])
torch.Size([868400, 3])
torch.Size([217100, 3])


(None, None, None, None)

In [ ]:
# del y_train
# del labels_encoded
# train_file.close()
# del train_file
# del train_data
# del y_train_tensor
# del X_train_tensor
# del train_loader
# del X_batch
# del y_batch

In [ ]:
import gc
gc.collect()

0

In [ ]:
# Initialize the model
learning_rate = 0.001
num_epochs = 10
input_dim = X_train_tensor.shape[2]
hidden_dim = 32 #64 
output_dim = y_train_tensor.shape[1]
num_models = 10

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

def train_model():      
  # Evaluate each model for 10 epochs
  for i in range(num_models):
      print('Model no', i+1)
      #Version 2
      # Create PyTorch DataLoader
      train_dataset = TensorDataset(X_train_subsets[i], y_train_subsets[i])
      train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
      
      # Initialize the model
      model = LSTM(input_dim, hidden_dim, output_dim).to(device)

      # Define the optimizer and scheduler for the current model
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
      scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

      # Checkpointing parameters for the current model
      checkpoint_interval = 1
      checkpoint_file = f'lstm_checkpoint_{i}.pth'

      #Version 2
      # Define early stopping parameters
      best_loss  = float('inf')
      early_stop_counter = 0
      early_stop_patience  = 3

      # Train the current model for 10 epochs
      for epoch in range(num_epochs):
          # Set model to train mode
          model.train()

          # Iterate over batches
          for x_batch, y_batch in tqdm(train_dataloader):
              # Move data to device
              x_batch, y_batch = x_batch.to(device), y_batch.to(device)

              # Forward pass
              outputs = model(x_batch)
              loss = criterion(outputs, y_batch)

              # Backward and optimize
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

          

          # Print the loss for every 10 epochs
          if (epoch+1) % 10 == 0:
              print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

          # Check if current model has reached early stopping criteria
          print(loss.item())
          if loss.item() < best_loss:
              best_loss = loss.item()
              early_stop_counter = 0
          else:
              early_stop_counter += 1

          if early_stop_counter >= early_stop_patience:
              print(f"Training for model {i} stopped early at epoch {epoch+1} due to early stopping")
              break

          # Save checkpoint at regular intervals
          if (epoch+1) % checkpoint_interval == 0:
              torch.save({
                  'epoch': epoch+1,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'loss': loss.item(),
              }, checkpoint_file)

          # Step the scheduler
          scheduler.step()        

        
      # Delete variables to free up memory
      del model, optimizer, scheduler, train_dataset

      # Empty the cache to free up GPU memory
      torch.cuda.empty_cache()
  return    

In [ ]:
X_train_subsets, y_train_subsets = create_subset(X_train_tensor, y_train_tensor, num_models)
train_model()

Model no 1


100%|██████████| 34/34 [00:01<00:00, 27.79it/s]


1.081726312637329


100%|██████████| 34/34 [00:00<00:00, 359.94it/s]


1.0920677185058594


...

1.1018667221069336


100%|██████████| 34/34 [00:00<00:00, 358.47it/s]


1.1034066677093506


100%|██████████| 34/34 [00:00<00:00, 377.66it/s]

1.091912865638733
Training for model 9 stopped early at epoch 9 due to early stopping


In [ ]:
def validation_call(X_data=None, y_data=None, model=None, isValidation=True, data_generator=None):
  # Evaluate the current model on the training set
  cnt = 0
  for X_data, y_data in data_generator:    
    with torch.no_grad():
      X_data = X_data.float()
      outputs = model.forward(X_data.to(device))          

    # Convert predicted and true labels to one-hot encodings
    predicted_labels = torch.argmax(outputs, dim=1).to(device)
    if isValidation:
      predictions.append(predicted_labels)
    true_labels = torch.argmax(y_data, dim=1).to(device)

    # Calculate precision for each class
    precisions = []
    accuracies = []
    for i in range(3):
      # if i == 2: # positive class
      #     true_positives = torch.sum((predicted_labels == i) & (true_labels == i))
      #     false_positives = torch.sum((predicted_labels == i) & (true_labels != 2))
      #     precision = true_positives.float() / (true_positives + false_positives).float()
      #     accuracies.append(torch.mean((predicted_labels == true_labels).float() * (true_labels == i).float()))
      # else: # negative class
      #     true_negatives = torch.sum((predicted_labels == i) & (true_labels == i) & (true_labels != 2))
      #     false_positives = torch.sum((predicted_labels == i) & (true_labels != i) & (true_labels != 2))
      #     precision = true_negatives.float() / (true_negatives + false_positives).float()
      #     accuracies.append(torch.mean((predicted_labels == true_labels).float() * (true_labels != i).float()))
      # precisions.append(precision)

        true_positives = torch.sum((predicted_labels == i) & (true_labels == i))
        false_positives = torch.sum((predicted_labels == i) & ((true_labels != i) & (true_labels != 3)))
        precision = true_positives.float() / (true_positives + false_positives).float()
        if torch.isnan(true_positives) or torch.isnan(false_positives) or torch.isnan(precision):
          if cnt == 0:
            print(i, predicted_labels, true_labels, true_positives, false_positives)
          cnt += 1
        precisions.append(precision)
        accuracy = torch.mean((predicted_labels == true_labels).float() * (true_labels == i).float())
        accuracies.append(accuracy)

    # Calculate average precision and accuracy
    avg_precision = torch.mean(torch.tensor(precisions))
    avg_accuracy = torch.mean(torch.tensor(accuracies))

    # Calculate percentage of positive predictions
    percent_positive = torch.mean((predicted_labels == 2).float())
    # Free Memory
  del model
  torch.cuda.empty_cache()
  return [avg_precision, avg_accuracy, percent_positive]

In [ ]:

predictions = []
def perform_validation(num_models, data_generator):
  metrics = []
  # Evaluate each model on the primary training set and the validation set
  for i in range(num_models):
    # print(f"\nModel {i+1}")
    # Load the trained model from the checkpoint file
    checkpoint_file = f'lstm_checkpoint_{i}.pth'
    checkpoint = torch.load(checkpoint_file)
    print("Model", i)
    model = LSTM(100, 16, 3).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    curr = {"train": [], "val": []}
    # curr["train"] = validation_call(X_train_subsets[i], y_train_subsets[i], model, False, data_generator)
    curr["val"] = validation_call(None, None, model, True, data_generator)
    metrics.append(curr)
    
  return metrics

In [ ]:
def print_table(metrics):
    header = ['Model', 'Train', '', '', 'Validation', '', '', '', '']
    subheader = ['', '', 'Precision', 'Accuracy', '% Positive', 'Precision', 'Accuracy', '% Positive']
    print(" {:<10} {:<10} {:<14} {:<10} {:<10} {:<14} {:<14} {:<14}".format(*header))
    print("{:<10} {:<0} {:<10} {:<10} {:<14} {:<10} {:<10} {:<14}".format(*subheader))

    for i in range(len(metrics)):
        row =[i+1] + [f"{val:.4f}" for val in metrics[i]['train']] + [f"{val:.4f}" for val in metrics[i]['val']]
        print("{:<11} {:<10} {:<10} {:<14} {:<10} {:<10} {:<14}".format(*row))
    return

In [ ]:
#Load Feature Engineered Stock dataset: 100 Features including labels
with open('data_with_features_clean_test.pkl', 'rb') as test_file: #To do: Change file name.
  test_data = pickle.load(test_file)

test_data = test_data.iloc[:200*stock_len]

In [ ]:
# Concatenate tensors of empty(Provding early shape can provide performance improvement)
start = 0
# if start == 0:
# X_train_tensor = torch.empty((batch_size*160, seq_length, 100), dtype=torch.float32)
# y_train_tensor = torch.empty((batch_size*160, 3), dtype=torch.float32)
generator = prepare_train_data(test_data, seq_length, batch_size)

In [ ]:
metrics

NameError: ignored

In [ ]:
# X_val_subsets, y_val_subsets = create_subset(X_test_tensor, y_test_tensor, num_models)
metrics = perform_validation(1, generator)
print_table(metrics)

Model 0


<ipython-input-5-d26e89e12f47>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(labels_encoded).float().clone().detach()


0 tensor([2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1,
        2, 1, 1, 1, 1, 1, 2, 1], device='cuda:0') tensor([0, 0, 1, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0,
        2, 2, 2, 2, 2, 0, 0, 2], device='cuda:0') tensor(0, device='cuda:0') tensor(0, device='cuda:0')
 Model      Train                                Validation                                             
            Precision  Accuracy   % Positive     Precision  Accuracy   % Positive    


IndexError: ignored

In [ ]:
print(metrics)

[{'train': [], 'val': [tensor(0.0833), tensor(0.0417), tensor(0.3750, device='cuda:0')]}]


In [ ]:
start = 0
ans = []
X_test_tensor = torch.empty((batch_size*5, seq_length, 100), dtype=torch.float32)
y_test_tensor = torch.empty((batch_size*5, 3), dtype=torch.float32)
for X_batch, y_batch in generator:
  print(type(X_batch), type(y_batch), X_batch.shape, y_batch.shape, X_batch.shape[0], y_batch.shape[0])
  end = start + X_batch.shape[0]
  X_test_tensor[start:end] = X_batch
  y_test_tensor[start:end] = y_batch
  start = end

In [ ]:
seq_length = 100 # Define sequence length
batch_size = 2201 - seq_length # Create data loader

# Prepare the data
generator = prepare_train_data(test_data, seq_length, batch_size)

seq_length = 30 # Define sequence length
batch_size = 32 # Create data loader

# # Prepare the data
X_test_tensor, y_test_tensor = prepare_train_data(test_data, seq_length, batch_size)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import IncrementalPCA
from sklearn.feature_selection import SelectKBest, f_regression

# Concatenate all dataframes in train_data into a single dataframe
df = train_data

# Separate the target variable (stock price) from the features
y = df['Label']
X = df.drop(['Label'], axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Instantiate IncrementalPCA and fit to the training data in chunks
batch_size = 1000
n_components = 20
ipca = IncrementalPCA(n_components=n_components, batch_size=batch_size)
for i in range(0, len(X_train), batch_size):
    X_train_batch = X_train.iloc[i:i+batch_size]
    ipca.partial_fit(X_train_batch)

# Transform the training and testing data in chunks
X_train_pca = []
for i in range(0, len(X_train), batch_size):
    X_train_batch = X_train.iloc[i:i+batch_size]
    X_train_pca_batch = ipca.transform(X_train_batch)
    X_train_pca.append(X_train_pca_batch)
X_train_pca = np.vstack(X_train_pca)

X_test_pca = []
for i in range(0, len(X_test), batch_size):
    X_test_batch = X_test.iloc[i:i+batch_size]
    X_test_pca_batch = ipca.transform(X_test_batch)
    X_test_pca.append(X_test_pca_batch)
X_test_pca = np.vstack(X_test_pca)

# # Instantiate PCA and fit to the training data
# pca = PCA(n_components=20)
# pca.fit(X_train)

# # Transform the training and testing data
# X_train_pca = pca.transform(X_train)
# X_test_pca = pca.transform(X_test)

# Instantiate SelectKBest and fit to the training data
selector = SelectKBest(f_regression, k=10)
selector.fit(X_train_pca, y_train.cat.codes.astype('int').to_numpy())

# Transform the training and testing data
X_train_selected = selector.transform(X_train_pca)
X_test_selected = selector.transform(X_test_pca)

selected_indices = selector.get_support(indices=True)

# Select only the columns with the selected indices
X_train_pca_df = pd.DataFrame(data=X_train_selected, columns=X_train.columns[selected_indices])
X_test_pca_df = pd.DataFrame(data=X_test_selected, columns=X_train.columns[selected_indices])

# Assign labels to X_train_pca_df
X_train_pca_df['Label'] = y_train.values
X_test_pca_df['Label'] = y_test.values

train_data = pd.concat([X_train_pca_df, X_test_pca_df])

# Print the first row of the original training data and the PCA-transformed data
print(df.iloc[0])
print(X_train_pca_df.iloc[0])

Open                                        0.672106
High                                        0.674597
Low                                         0.670546
Close                                       0.672106
Volume                                      0.208023
                                           ...      
linear_reg_forecast (LRF)                   0.689504
mass_index (Mass_Index)                     1.004993
median_price (Median Price)                 0.672572
money_flow_index (Money Flow Index)        27.384836
Label                                  no big change
Name: 1, Length: 101, dtype: object
High                                         4.574482
Close                                       23.298265
awesome_oscillator (AO)                    -53.453593
bollinger_percent_b (percent_b)            -31.491068
cmo (cmo)                                   -6.443804
disparity_index (disparity_index)           16.613215
elder_impulse_system (elder_bullish)        28.448544
eld

In [ ]:
del X_train_pca_df, X_train_selected, X_train_pca, X_train_batch
del X_test_pca_df, X_test_selected, X_test_pca, X_test_batch

In [ ]:
seq_length = 30 # Define sequence length
batch_size = 16 # Create data loader

# Prepare the data
X_train_tensor, y_train_tensor = prepare_train_data(train_data, seq_length, batch_size)

In [ ]:
from collections import Counter
final_predictions = [Counter(p).most_common(1)[0][0] for p in zip(*predictions)]
# final_predictions

In [ ]:
final_predictions.shape

torch.Size([21710])

In [ ]:
 # Calculate precision for each class
precisions = []
accuracies = []
# final_predictions = torch.argmax(final_predictions, dim=1).to(device)
true_labels = torch.argmax(y_val_subsets[0], dim=1).to(device)
for i in range(3):
  true_positives = torch.sum((final_predictions == i) & (true_labels == i))
  false_positives = torch.sum((final_predictions == i) & ((true_labels != i) & (true_labels != 3)))
  precision = true_positives.float() / (true_positives + false_positives).float()
  precisions.append(precision)
  accuracy = torch.mean((final_predictions == true_labels).type(torch.FloatTensor) * (true_labels == i).type(torch.FloatTensor))
  accuracies.append(accuracy)
# Calculate average precision and accuracy
avg_precision = torch.mean(torch.tensor(precisions))
avg_accuracy = torch.mean(torch.tensor(accuracies))

# Calculate percentage of positive predictions
percent_positive = torch.mean((final_predictions == 2).float())

print("Precision", avg_precision)
print("Precision", avg_accuracy)
print("Precision", percent_positive)

Precision tensor(0.7090)
Precision tensor(0.2365)
Precision tensor(0.2800, device='cuda:0')


In [ ]:
#Step 5
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

# Load the saved LSTM models and their predictions on the primary training and validation sets
models = []
preds_train = []
preds_val = []
for i in range(10):
    model = torch.load(f'lstm_checkpoint_{i}.pth')
    # preds_train.append(np.load('lstm_preds_train_' + str(i) + '.npy'))
    # preds_val.append(np.load('lstm_preds_val_' + str(i) + '.npy'))
    models.append(('lstm_' + str(i), model))

# Voting ensemble
voting_ensemble = VotingClassifier(models, voting='hard')
voting_ensemble.fit(np.concatenate(preds_train, axis=1), y_train)
train_preds_voting = voting_ensemble.predict(np.concatenate(preds_train, axis=1))
val_preds_voting = voting_ensemble.predict(np.concatenate(preds_val, axis=1))

# # Blending ensemble
# blend_train = np.concatenate([p.reshape(-1, 1) for p in preds_train], axis=1)
# blend_val = np.concatenate([p.reshape(-1, 1) for p in preds_val], axis=1)
# blend_model = LogisticRegression()
# blend_model.fit(blend_train, y_train)
# train_preds_blend = blend_model.predict(blend_train)
# val_preds_blend = blend_model.predict(blend_val)

# # Adaboost ensemble
# adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10)
# adaboost.fit(np.concatenate(preds_train, axis=1), y_train)
# train_preds_adaboost = adaboost.predict(np.concatenate(preds_train, axis=1))
# val_preds_adaboost = adaboost.predict(np.concatenate(preds_val, axis=1))

# # Compute evaluation metrics
# train_acc_voting = accuracy_score(y_train, train_preds_voting)
# train_acc_blend = accuracy_score(y_train, train_preds_blend)
# train_acc_adaboost = accuracy_score(y_train, train_preds_adaboost)
# val_acc_voting = accuracy_score(y_val, val_preds_voting)
# val_acc_blend = accuracy_score(y_val, val_preds_blend)
# val_acc_adaboost = accuracy_score(y_val, val_preds_adaboost)
# train_f1_voting = f1_score(y_train, train_preds_voting, average='macro')
# train_f1_blend = f1_score(y_train, train_preds_blend, average='macro')
# train_f1_adaboost = f1_score(y_train, train_preds_adaboost, average='macro')
# val_f1_voting = f1_score(y_val, val_preds_voting, average='macro')
# val_f1_blend = f1_score(y_val, val_preds_blend, average='macro')
# val_f1_adaboost = f1_score(y_val, val_preds_adaboost, average='macro')

# # Print the evaluation metrics
# print('Method\tDataset\tAccuracy\tF1-score')
# print('Voting\tTrain\t{}\t{}'.format(train_acc_voting, train_f1_voting))
# print('Voting\tVal\t{}\t{}'.format(val_acc_voting, val_f1_voting))
# print('Blending\tTrain\t{}\t{}'.format(train_acc_blend, train_f1_blend))
# print('Blending\tVal\t{}\t{}'.format(val_acc_blend, val_f1_blend))
# print('Adaboost\tTrain\t{}\t{}'.format(train_acc_adaboost, train_f1_adaboost))
# print('Adaboost\tVal\t{}\t{}'.format


In [ ]:
import torch
import torch.nn as nn

class Seq2Seq(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2Seq, self).__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, output_dim, batch_first=True)
        
    def forward(self, x):
        # Encoder
        _, (hidden, cell) = self.encoder(x)
        
        # Decoder
        batch_size, sequence_length, _ = x.size()
        output = torch.zeros(batch_size, sequence_length, self.decoder.hidden_size).to(x.device)
        h = hidden
        c = cell
        for t in range(sequence_length):
            out, (h, c) = self.decoder(output[:, t:t+1, :], (h, c))
            output[:, t, :] = out.squeeze()
        
        return output


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import multiprocessing

class MyDatasetSuper(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y        
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

class MyDataset(Dataset):
    def __init__(self, df, seq_length):
        self.X = []
        self.y = []
        
        labels = df.iloc[:, -1].values # Extract labels from the last column
        cnt = len(labels)
        print(cnt)
        df = df.iloc[:, :-1] # Remove labels column from the DataFrame        
        values = torch.tensor(df.values) # Convert DataFrame to tensor

        # Create input/output sequences
        while cnt != 0:
          for i in range(seq_length, stock_len):
            self.X.append(values[i-seq_length:i, :])
            self.y.append(labels[i])
          cnt -= 2201
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

def data_generator(train_data, seq_length, batch_size, num_workers):
    train_dataset = MyDataset(train_data, seq_length)

    # Create DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    for X_batch, y_batch in train_loader:
        # Convert labels to tensor
        y_train = np.array(y_batch)
        labels_encoded = torch.zeros((len(y_train), 3), dtype=torch.float32)
        labels_encoded[y_train == 'increase', 2] = 1
        labels_encoded[y_train == 'no big change', 1] = 1
        labels_encoded[y_train == 'decrease', 0] = 1
        y_train_tensor = torch.tensor(labels_encoded).float().clone().detach()

        yield X_batch, y_train_tensor

    # Free up memory
    del train_dataset
    del labels_encoded
    del X_batch, y_batch

def load_data_in_chunks(filename, chunk_size):
    with open(filename, 'rb') as f:
        while True:
            try:
                chunk = []
                for i in range(chunk_size):
                    chunk.append(pickle.load(f))
                yield chunk
            except EOFError:
                break

# def prepare_train_data(train_data = None, seq_length = 100, batch_size = 32):
num_workers=multiprocessing.cpu_count() #parallel computing

# Optimization 4 (Using data generator)
generator = data_generator(train_data, seq_length, batch_size, num_workers)

# Iterate over data chunks and create the dataset
# X = []
# y = []
# for chunk in data_chunks:
#     for i in chunk:
#       train_dataset = MyDataset(i, seq_length)
#       X.extend(train_dataset.X)
#       y.extend(train_dataset.y)
#       del train_dataset        
  
# train_dataset = MyDataset(train_data, seq_length)
# train_dataset = MyDatasetSuper(X, y)

#Optimzation 1: Dataset and Dataloaders for batch processing
#Optimization 2: (num_of_workers, pin_memory(GPU specific))
# Create DataLoader

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

# Concatenate tensors of empty(Provding early shape can provide performance improvement)
# X_train_tensor = torch.empty((len(train_dataset), seq_length, train_dataset[0][0].shape[1]), dtype=torch.float32)

# #Optimization 3: Avoid torch.cat on large datasets.
# start = 0
# for X_batch, y_batch in train_loader:
#     end = start + X_batch.shape[0]
#     X_train_tensor[start:end] = X_batch
#     start = end

# # Convert labels to tensor
# y_train = np.array(train_dataset.y)
# labels_encoded = torch.zeros((len(y_train), 3), dtype=torch.float32)
# labels_encoded[y_train == 'increase', 2] = 1
# labels_encoded[y_train == 'no big change', 1] = 1
# labels_encoded[y_train == 'decrease', 0] = 1
# # y_train_tensor = torch.tensor(labels_encoded).float() #Warning Issued
# y_train_tensor = torch.tensor(labels_encoded).float().clone().detach()

# # Free up memory
# del train_dataset
# del labels_encoded
# del start, end, seq_length, num_workers, X_batch, y_batch

  # return X_train_tensor, y_train_tensor

In [ ]:
# Define the split point
split_point = int(len(X_train_tensor) * 0.8) # 80% train, 20% validation

# Split the data into training and validation sets
X_train_tensor = X_train_tensor.float()
X_train_split = X_train_tensor[:split_point]
y_train_split = y_train_tensor[:split_point]
X_val_split = X_train_tensor[split_point:]
y_val_split = y_train_tensor[split_point:]